## Import

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Nov 12 14:57:47 2020

@author: yuhanzhang
"""



yago_site = "https://yago-knowledge.org/sparql/query"
yago_code_part1 = '''PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
select ?property_name ?value_name
(count(?city) as ?cnt)
where {
        <http://yago-knowledge.org/resource/'''
yago_code_part2 =  '''> ?property ?valueOrObject .
       ?city ?property ?valueOrObject.
   		?property rdfs:label ?property_name.
 	 ?valueOrObject rdfs:label ?value_name.
  	filter(lang(?value_name) = 'en').
      } 
group by?property_name ?value_name
HAVING (?cnt > 1)
'''


yago_inversecode_part1 = '''
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
select  ?property_name ?un_name
#(group_concat(?un_name; separator = ';') as ?members)
#(count(?un) as ?cnt)
where {
       ?un  ?property <http://yago-knowledge.org/resource/'''
yago_inversecode_part2 = '''> .
       ?property rdfs:label ?property_name.
       ?un rdfs:label ?un_name.
       filter(lang(?un_name) = 'en').

      } 
#group by ?property_name
'''

yago_query_country = '''
PREFIX schema: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
                    select  ?itemLabel 
(group_concat(?name_orgranization; separator = ';') as ?memberOf)
                    where {
                            ?country rdf:type schema:Country.
  							?country rdfs:label ?itemLabel.
  							filter ( lang(?name_country) = 'en').
							?country schema:memberOf ?member.
  ?member rdfs:label ?name_orgranization.
  filter(lang(?name_orgranization)='en').
                            } 
group by ?itemLabel
limit 50
'''

yago_country_properties = '''

'''                    
                    
stopwords = ['image', '2000/01/rdf-schema#comment','2000/01/rdf-schema#label','alternateName', 'sameAs', 'url']
             
conceptnet_site_basic1 = 'http://api.conceptnet.io/c/en/'
conceptnet_site_basic2 = '?filter=/c/en'

conceptnet_site_related1 = 'http://api.conceptnet.io/related/c/en/'
conceptnet_site_related2 = '?filter='

conceptnet_site_relation1 = 'http://api.conceptnet.io/relatedness?node1=/c/en/'
conceptnet_site_relation2 = '&node2=/c/en/'

dbpedia_site = "http://dbpedia.org/sparql"
dbpedia_code_part1 = """
                        select ?un ?a where{
                        
                                ?un rdf:type <http://dbpedia.org/ontology/City>.
                                ?un <http://dbpedia.org/ontology/country> <http://dbpedia.org/resource/China>.
                                ?un <http://dbpedia.org/ontology/"""
dbpedia_code_part2 = """> ?a.
                                }
                        """

wikidata_query_site = "https://query.wikidata.org/sparql"
wikidata_data_site = 'https://en.wikipedia.org/w/api.php?action=query&prop=pageprops&ppprop=wikibase_item&redirects=1&titles=Elvis_Presley'
wikidata_query_city = '''
SELECT distinct ?item ?itemLabel ?length_in_m ?area_in_m2 ?population ?itemdesc
WHERE
{
  ?item          wdt:P31/wdt:P279*           wd:Q515.    # city
  ?item          p:P2044                     ?stmnode.    # sealevel
  ?stmnode       psv:P2044                   ?valuenode.
  ?valuenode     wikibase:quantityAmount     ?length.
  ?valuenode     wikibase:quantityUnit       ?unit.
  
  ?item          p:P2046                     ?area.
  ?area          psv:P2046                   ?areavalue.
  ?areavalue     wikibase:quantityAmount     ?areabig.
  ?areavalue     wikibase:quantityUnit       ?unitarea.

  ?item          wdt:P1082                   ?population.
  
  ?item schema:description ?itemdesc.
  FILTER(LANG(?itemdesc) = "en")

  # conversion to SI unit
  ?unit          p:P2370                 ?unitstmnode.   # conversion to SI unit
  ?unitstmnode   psv:P2370               ?unitvaluenode. 
  ?unitvaluenode wikibase:quantityAmount ?conversion.
  ?unitvaluenode wikibase:quantityUnit   wd:Q11573.      # meter
  
  BIND(?length * ?conversion AS ?length_in_m).
  
  ?unitarea          p:P2370       ?unitareastmode.
  ?unitareastmode    psv:P2370     ?unitareavaluenode.
  ?unitareavaluenode wikibase:quantityAmount ?conversionarea.
  ?unitareavaluenode wikibase:quantityUnit  wd:Q25343.
  
   BIND(?areabig * ?conversionarea AS ?area_in_m2).
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} 
    '''
wikidata_city_properties = ['area','population','sea_level']


google_site_part1 = "https://www.google.com/search?q="

In [2]:
!pip install SPARQLWrapper

     |████████████████████████████████| 235kB 10.7MB/s 
     |████████████████████████████████| 51kB 7.6MB/s 


In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sat Dec 12 17:03:24 2020

@author: yuhanzhang

This python file provides methods to generate pertinent descriptions based on knowledge bases.
"""


import numpy as np
from SPARQLWrapper import SPARQLWrapper, JSON
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
import requests
import pandas as pd
import sys
import re
import math
from bs4 import BeautifulSoup
import time

class SparQL:
    '''
    This class realizes those functions: 
        1.send queries to knowledge bases and get result; 
        2.organize information in pandas data frame
        3.calculate complexity of information
        4.generate pertinent descriptions from complexity
        
    objects in this class:
        1.searching_objet: the item one interested in, in format str
        2.query: query to send to obtain normal properties, in format str
        3.inverse_query: query to send to obtain incoming properties, in format str
        4.endpoint: knowledge base's endpoint, in format str
    '''

    
    def get_results(self, inverse):
        '''
        Get results from knowledge bases.
        ----------
        input: self
               Boolean value: inverse. If inverse = Flase, get results of normal properties; else, get results of incoming properties
        ----------
        output: a dictionary with unorganized information
        '''
        user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
        # TODO adjust user agent; see https://w.wiki/CX6
        sparql = SPARQLWrapper(self.endpoint, agent=user_agent)
        
        # To collect incoming properties
        if inverse:
            sparql.setQuery(self.inverse_query)
        # To collect normal properties
        else:
            sparql.setQuery(self.query)
        sparql.setReturnFormat(JSON)       
        return sparql.query().convert()
    
    def wikidata_query(self, inverse):
        '''
        Get organized information
        ----------
        input: self
               Boolean value: inverse. If inverse = Flase, get results of normal properties; else, get results of incoming properties
        ----------
        output: a dataframe with organized information
        '''
        results = self.get_results(inverse)
        if len(results['results']['bindings']) > 0:
            keys = list(results['results']['bindings'][0].keys())
        else:
            proto = []
            df = pd.DataFrame(proto)
            return df
        prototype = []
        flag = 0
        for item in results['results']['bindings']:
            #if self.searching_objet != item['value_name']['value']:
            if True:
              dictionary = {}
              for key in keys:
                  if item[key]['value'] in self.searching_objet:
                      flag = -1
                  try:
                      dictionary[key] = float(item[key]['value'])
                  except:
                      dictionary[key] = item[key]['value']
              if flag != -1:        
                  prototype.append(dictionary)

        df = pd.DataFrame(prototype)
        return df
    
    def wiki_contrast(self):
        results = self.get_results(False)
        cities = []

        for item in results['results']['bindings']:
            cities.append({
                    #'area': item['area']['value'],
                    'itemLabel': item['itemLabel']['value'],
                    'itemdesc': item['itemdesc']['value'],
                    'population': float(item['population']['value']),
                    'area': float(item['area_in_m2']['value']),
                    'sea_level': float(item['length_in_m']['value'])
                    })
        df = pd.DataFrame(cities)
        return df
    
    def find_hitsnumber(self,searching_item):
        '''
        Get hits number of one item in Google
        ----------
        input: searching_item(str): The item we want to search
        ----------
        output: hits number (int)
        '''        
        
        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:23.0) Gecko/20100101 Firefox/23.0'}
        # Accroding to the rule of URL of Google, the space will be replaced by the sign + in the url.
        searching = re.sub('\ ','+',searching_item)
        # For users from different countries, we use different urls
        if self.language == 'en': # English speakers
            response = requests.get("https://www.google.com/search?q=\""+searching+"\"", headers = headers)
        elif self.language == 'fr': # French
            #response = requests.get("https://www.google.com/search?q="+searching+"&hl=fr", headers = headers)
            response = requests.get("https://www.google.fr/search?q=\""+searching+"\"&lr=lang_fr", headers = headers)
        elif self.language == 'cn': # Chinese
            #response = requests.get("https://www.google.com.hk/search?q="+searching+"&lr=lang_zh-CN", headers = headers)
            response = requests.get("https://www.google.com.hk/search?q=\""+searching+"\"&lr=lang_zh-CN", headers = headers)
        
        time1 = time.time()
        res = response.text
        soup = BeautifulSoup(res, "lxml")
        time2 = time.time()
        #print(time2-time1)
        str_hits = soup.find_all(attrs={'id':'result-stats'})
        l= re.split(r'[\s\,]+',str(str_hits))
        hits = ''
        for i in range(len(l[2:-5])):
            hits += l[2:-5][i]
        try:
            hit_number = int(hits)
        except:
            hit_number = 0
        #print(str_hits)
        return hit_number

    def relatedness_conceptnet(self, item1, item2):
        '''
        Get relatedness between two items
        ----------
        input: item1, item2
        ----------
        output: the relatedness calculated based on ConceptNet, which is a cosine similarity
        '''
        try:
            i1 = item1.lower()
            i2 = item2.lower()
        except:
            return 0
        sujet1 = re.sub('\ ','_',i1)
        sujet2 = re.sub('\ ','_',i2)
        try:
          response = requests.get('http://api.conceptnet.io/relatedness?node1=/c/en/'+sujet1+'&node2=/c/en/'+sujet2)
          obj = response.json()
          relatedness = abs(obj['value'])
        except:
          relatedness = 0
        return relatedness
    

    
    def calculate_complexity_1(self, context):
        '''
        Get complexity of normal properties
        ------------
        input: context (list)
        ------------
        output: complexity (list), data
        
        ------------
        Here we calculate the conditional complexity. We apply the chain rule to calculate the complexity:
            Complexity(A, value) = Complexity(A|value) + Complexity(value)
            For example, if one description is: China is a member of G20, then value is G20, A is China.
            Complexity(China,G20) = Complexity(China|G20) + Complexity(G20), 
            Since there are 20 members in G20, thus, when G20 is known, Find China is like find one element in a list of 20 elements
            As a result, Complexity(China|G20) = log2(20)
        What's more, Complexity(value) could be calculated by using knowledge base or only context, here we use the number of hits on Google.
        Since the cost of using knowledge base is more expensive, we give it 10 more bits for its complexity.
        '''
        df = self.wikidata_query(False)
        if len(df) == 0:
            return False, df
        complexity_value = []
        relatednesses = []
        contexte = np.array(context)
        lllll = np.zeros(len(df))
        for i in range(len(df)):
            if df['value_name'][i] not in context:
              if self.relatedness_conceptnet(self.searching_objet, df['value_name'][i]) > 0.1:
                relatedness = self.find_hitsnumber( df['value_name'][i])
              else:
                relatedness = 0
                
            else:
                relatedness = 0
            relatednesses.append({'relatedness': float(relatedness),
                          'value':df['value_name'][i],
                          'index':i})
        temp_relatedness = pd.DataFrame(relatednesses)
        sorted_relatedness = temp_relatedness.copy()
        sorted_relatedness.sort_values(by='relatedness',inplace=True,ascending = False)
        orders = list(range(len(temp_relatedness)))
        for i in range(len(df)):
            lllll[list(sorted_relatedness['index'])[i]] =  math.log(orders[i]+1,2)+ math.log(df['cnt'][i],2)    

        for i in range(len(df)):
            if df['value_name'][i] in context:
                temp_com = math.log(len(context) - np.argwhere(contexte == df['value_name'][i])[0][0])
                lllll[i] = min(temp_com, lllll[i])
        return lllll, df
    
    def calculate_complexity_22(self, context):
        '''
        Get complexity of incoming properties
        ------------
        input: context (list)
        ------------
        output: complexity (list), data
        
        '''
        df1 = self.wikidata_query(True)
        #print(df1)
        dropped = []
        no_use = ['death place', 'birth place', 'home location', 'Metro','tram','station','Station','rue','Rue']
        for item in no_use:
            try:
                dropped += [i for i,v in enumerate(list(df1['property_name'])) if v==item]
                dropped += [i for i,v in enumerate(list(df1['un_name'])) if item in v]
            except:
                pass
        df2 = df1.drop(dropped)
        df = df2.sample(n = min(20,len(df2)), random_state = None)
        df.reset_index(drop=True, inplace=True)
        complexity_value = []
        relatednesses = []
        contexte = np.array(context)
        lllll = np.zeros(len(df))
        
        for i in range(len(df)):
            if df['un_name'][i] not in context:
              #Because sometimes the incomming properties could be very general, like metro line 4, line 9
              #So we limit the corresponding properties should at least have link with demanded city
              if self.relatedness_conceptnet(self.searching_objet, df['un_name'][i]) > 0.1:
                relatedness = self.find_hitsnumber( df['un_name'][i])
              else:
                relatedness = 0
                
            else:
                relatedness = 0
            relatednesses.append({'relatedness': float(relatedness),
                          'value':df['un_name'][i],
                          'index':i})
        temp_relatedness = pd.DataFrame(relatednesses)
        sorted_relatedness = temp_relatedness.copy()
        sorted_relatedness.sort_values(by='relatedness',inplace=True,ascending = False)
        orders = list(range(len(temp_relatedness)))
        for i in range(len(df)):
            lllll[list(sorted_relatedness['index'])[i]] =  math.log(orders[i]+1,2)    

        for i in range(len(df)):
            if df['un_name'][i] in context:
                temp_com= math.log(len(context) - np.argwhere(contexte == df['un_name'][i])[0][0])
                lllll[i] = min(temp_com, lllll[i])
                #lllll[i] = 0
        return lllll, df

    
    '''
    Function to generate descriptions based on complexity
    '''
    def DescriptionByComplexity(self,number,context):
        complexity, datas = self.calculate_complexity_1(context)
        #datas = self.wikidata_query()
        if len(datas) == 0:
            print('No enough data')
            return 'False', 'False'
        descriptions = []
        valeurs = []
        #complexities = []
        temp = np.array(complexity)
        large_number = temp.argsort()[::1]
        for i in range(len(large_number)):
            prop = datas['property_name'][large_number[i]]
            valeur = datas['value_name'][large_number[i]]
            pertinence = temp[large_number[i]]
            description = self.searching_objet + '----'+str(prop)+'---->'+str(valeur)+'   with complexity '+str(pertinence)
            descriptions.append(description)
            valeurs.append(valeur)
        return descriptions[:number], valeurs[:number]
    
    def DescriptionByComplexity_inverse2(self,number,context):
        complexity, datas = self.calculate_complexity_22(context)
        #datas = self.wikidata_query()
        descriptions = []
        valeurs = []
        temp = np.array(complexity)
        large_number = temp.argsort()[::1]
        for i in range(len(large_number)):
            prop = datas['property_name'][large_number[i]]
            valeur = datas['un_name'][large_number[i]]
            pertinence = temp[large_number[i]]
            description = self.searching_objet + '----'+str(prop)+'---->'+str(valeur)+'   with complexity '+str(pertinence)
            descriptions.append(description)
            valeurs.append(valeur)
        return descriptions[:number], valeurs[:number]


    '''
    Three functions for generating descriptions with quantitative properties
    ''' 
    def get_order(self,attribu, data, article):
        """
        Input: 1. attribu: Features that users want to sort, str
               2. data: DataFrame that contains all the information, DataFrame
               3. article: City that users want to find the ranking
        Output: order: City's ranking with respect to chosen feature
        """
        temp = data.copy()
        # To find which column stores the names
        name_column = temp.columns.values.tolist().index('itemLabel')
        # When given a feature, sort the datas
        temp.sort_values(by=attribu,inplace=True,ascending = False)
        order = np.argwhere(temp.values[:,name_column]==article)[0][0]+1
        return order

    def intersting_property(self, attribus):
        '''
        Input: attribus: all the features that the user want to take into consideration
        Output: Corresponding descriptions
        '''
        data = self.wiki_contrast()
        orders = []
        complexities = []
        items = []
        for item in attribus:
            if item != 'itemLabel':
              ranking = self.get_order(item, data, self.searching_objet)
              orders.append(ranking)
              # Population ranks 716, area ranks 296 and altitude ranks 6915 for words frequency
              if item == 'population':
                complexities.append(math.log(ranking,2) + math.log(716,2))
              elif item == 'sea_level':
                complexities.append(math.log(ranking,2) + math.log(6915,2))
              else:
                complexities.append(math.log(ranking,2) + math.log(296,2))
              items.append(item)
        complexity = min(complexities)
        order = min(orders)
         # The smallest ranking is the most interesting for human beings
        #order_index = orders.index(min(orders))
        order_index = complexities.index(min(complexities))
        name_column = data.columns.values.tolist().index('itemLabel')
        place = np.argwhere(data.values[:,name_column]==self.searching_objet)[0][0]
        description = str(data['itemdesc'][place])

        descrption = self.searching_objet+''' is the city who has the '''+ str(orders[order_index])+'''th largest '''+items[order_index]+ ''' among the world'''
        #print()
        return description, descrption
    
    def __str__(self):
        a = self.searching_objet
        b = self.query
        c = self.inverse_query
        d = self.endpoint
        return "The searching object is "+a+", the query is "+b+", the inverse query is "+c+" and the endpoint is "+d
    


class Yago(SparQL):
    '''
    This class is used to build an interface to visit Yago
    '''
    language = ''
    searching_objet = ''
    query = ''
    inverse_query = ''
    endpoint = yago_site
    context = []
    
    def __init__(self, searchobjet, language):
        self.language = language
        self.searching_objet = searchobjet
        self.query = yago_code_part1 + self.searching_objet + yago_code_part2
        self.inverse_query = yago_inversecode_part1 + self.searching_objet + yago_inversecode_part2
        #self.query = constant.yago_query_country


class Pertinent_Description():
  context = []

  def __init__(self):
    #a = input()
    self.context = []

  def Get_description(self, searching_object, language):
    searching = re.sub('_',' ',searching_object)
    y = Yago(searching_object,language)
    Descriptions,items = y.DescriptionByComplexity(5,self.context)
    Descriptions_in, items_in = y.DescriptionByComplexity_inverse2(5,self.context)
    
    w = Wikidata(searching_object)
    Contrast_description = w.intersting_property(['population','sea_level','area'])
    if Descriptions == 'False':
        return Descriptions_in, Contrast_description
    for item in items:
        if item not in self.context:
            self.context.append(item)
    for item_in in items_in:
        if item_in not in self.context:
            self.context.append(item_in)
    self.context.append(searching)
    return Descriptions, Descriptions_in, Contrast_description
    #return Descriptions_in



        
class Wikidata(SparQL):
    searching_objet = ''
    endpoint = wikidata_query_site
    query = ''

    
    def __init__(self, searching_objet):
        self.searching_objet = searching_objet
        self.query = wikidata_query_city
    '''    
    def data_range(self):
        data = self.wikidata_query()
        return data
    '''

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


# instructions

Please just press on the "running button" at the very beginning of each block. If you want to search for some other cities, for example: Beijing, just use the line:

l.Get_description('Beijing','cn')

## realization

In [4]:
l = Pertinent_Description()

In [20]:
l.Get_description('Nagqu','cn')

No enough data


(['Nagqu----contained in place---->Sog County   with complexity 0.0',
  'Nagqu----contained in place---->Tanggula Mountains   with complexity 1.0',
  'Nagqu----contained in place---->Amdo County   with complexity 1.5849625007211563',
  'Nagqu----contained in place---->Nyima County   with complexity 2.0',
  'Nagqu----location---->Battle of the Salween River   with complexity 2.321928094887362'],
 ("City in Tibet, People's Republic of China",
  'Nagqu is the city who has the 4th largest area among the world'))

In [5]:
l.Get_description('Chengdu','fr')

(['Chengdu----contained in place---->Sichuan   with complexity 10.161131882984307'],
 ['Chengdu----contained in place---->Chongzhou   with complexity 0.0',
  'Chengdu----contained in place---->Chengdu Normal University, CDNU   with complexity 1.0',
  'Chengdu----contained in place---->Line 5   with complexity 1.5849625007211563',
  'Chengdu----contained in place---->Southwest Jiaotong University   with complexity 2.0',
  'Chengdu----contained in place---->Qionglai City   with complexity 2.321928094887362'],
 ('the capital of Sichuan Province, China',
  'Chengdu is the city who has the 21th largest population among the world'))

In [6]:
l.Get_description('Paris','cn')

(['Paris----contained in place---->Metropolis of Greater Paris   with complexity 4.700439718141093',
  'Paris----award---->Companion of the Liberation   with complexity 6.491853096329675',
  'Paris----award---->Croix de guerre 1914–1918   with complexity 6.714245517666123',
  'Paris----location---->Metropolis of Greater Paris   with complexity 7.238404739325079',
  'Paris----location---->Île-de-France   with complexity 11.366322214245816'],
 ['Paris----content location---->I Am not an Easy Man   with complexity 0.0',
  'Paris----contained in place---->Hôtel de Monaco   with complexity 1.0',
  'Paris----contained in place---->Belleville   with complexity 1.5849625007211563',
  'Paris----location---->Stade Français   with complexity 2.0',
  'Paris----location---->Sorbonne Universities   with complexity 2.321928094887362'],
 ('capital and largest city of France',
  'Paris is the city who has the 261th largest population among the world'))

In [38]:
l.Get_description('Shanghai','cn')

(["Shanghai----contained in place---->People's Republic of China   with complexity 2.9444389791664403"],
 ['Shanghai----contained in place---->Lianqi He   with complexity 0.0',
  'Shanghai----location---->Shanghai Experimental School   with complexity 1.0',
  'Shanghai----contained in place---->Beicao Hangdao   with complexity 1.5849625007211563',
  'Shanghai----contained in place---->Chongming District   with complexity 2.0',
  'Shanghai----location---->Donghua University   with complexity 2.1972245773362196'],
 ('municipality of China',
  'Shanghai is the city who has the 2th largest population among the world'))

In [7]:
l.Get_description('Lille','cn')

(['Lille----contained in place---->canton of Lille-4   with complexity 3.321928094887362',
  'Lille----location---->arrondissement of Lille   with complexity 4.321928094887362',
  'Lille----contained in place---->canton of Lille-Nord-Est   with complexity 4.584962500721156',
  'Lille----contained in place---->arrondissement of Lille   with complexity 4.643856189774724',
  'Lille----location---->canton of Lille-3   with complexity 4.807354922057604'],
 ['Lille----contained in place---->Institut Régional d’Administration de Lille   with complexity 0.0',
  'Lille----contained in place---->Église Saint-Sauveur de Lille   with complexity 1.0',
  'Lille----location---->Lycée International Montebello   with complexity 1.5849625007211563',
  'Lille----contained in place---->Le Prato   with complexity 2.0',
  'Lille----location---->Feast of the Pheasant   with complexity 2.321928094887362'],
 ('commune in Nord, France',
  'Lille is the city who has the 1665th largest population among the world'

In [6]:
l.Get_description('Kunming','fr')

(['Kunming----contained in place---->Yunnan   with complexity 8.113742166049189'],
 ['Kunming----contained in place---->Xundian Hui and Yi Autonomous County   with complexity 0.0',
  'Kunming----contains place---->Yunnan   with complexity 1.0',
  'Kunming----location---->Yunnan University of Finance and Economics   with complexity 1.5849625007211563',
  'Kunming----contained in place---->Yunnan Normal University Business School   with complexity 2.0',
  'Kunming----location---->Kunming Tuodong Sports Center   with complexity 2.321928094887362'],
 ('Capital of Yunnan province, China',
  'Kunming is the city who has the 108th largest area among the world'))

In [31]:
l.Get_description('Paris','cn')

(['Paris----award---->Legion of Honour   with complexity 3.6635616461296463',
  'Paris----location---->Metropolis of Greater Paris   with complexity 4.700439718141093',
  'Paris----award---->Croix de guerre 1914–1918   with complexity 6.714245517666123',
  'Paris----contained in place---->Metropolis of Greater Paris   with complexity 7.238404739325079',
  'Paris----contained in place---->Île-de-France   with complexity 10.951284714966972'],
 ['Paris----content location---->Bel Ami   with complexity 0.0',
  'Paris----location---->Hôtel George-V   with complexity 1.0',
  'Paris----content location---->Danton   with complexity 1.5849625007211563',
  'Paris----location---->Théâtre du Soleil   with complexity 2.0',
  'Paris----location---->École de guerre   with complexity 2.321928094887362'],
 ('capital and largest city of France',
  'Paris is the city who has the 261th largest population among the world'))

In [ ]:
l.Get_description('Lyon','cn')
